In [5]:
import torch
import torchvision
import cv2
import numpy as np


In [6]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)

In [128]:
import math
from time import sleep
dis_threshold = 0.1
def convert_angle(delta_x,delta_y):
    if(delta_x == 0):      #special case when x = 0 
        if(delta_y == 0):          #reach the destination
            return None
        elif(delta_y > 0):     
            return 90
        else:
            return -90
    temp = delta_y / delta_x
    
    if(delta_x > 0): #in case from -90 to 90 degree
        return math.degrees(math.atan(temp))
    else:
        
        if(delta_y > 0):  #case for 90 to 180 degree
            return 180 + math.degrees(math.atan(temp))
        else:       #case for -180 to -90 degree
            return math.degrees(math.atan(temp)) + 180 - 360
        
        
def update_info_straight(curr_x,curr_y,curr_angle, time):    #for keep space
    speed = 0.3      #currently defalut 1, can use calibration to change that
    curr_x = curr_x + math.cos(math.radians(curr_angle)) * time * speed
    curr_y = curr_y + math.sin(math.radians(curr_angle)) * time * speed

    return curr_x,curr_y

def reach_destination(curr_x, x, curr_y, y):  
    distance_apart = math.sqrt((curr_x - x)**2 + (curr_y - y)**2)       #use to calculate the distance from the destination to current lcoation
    
    if(distance_apart < dis_threshold): #within a range, then it mean it is close to the destination, then it return true.
        return True
    else:
        return False

def turn_left(leftspeed = 0, rightspeed = 0.6):    # use for easy control of the car
    robot.set_motors(leftspeed, rightspeed)

def turn_right(leftspeed = 0.6, rightspeed = 0):    
    robot.set_motors(leftspeed, rightspeed)

def spin_left(speed = 0.5):
    robot.left(speed)

def spin_right(speed = 0.5):
    robot.right(speed)

def straight(speed = 0.3):
    robot.forward(speed)
    #robot.left_motor.value = speed
    #robot.right_motor.value = speed 

def check_angle(angle):     #use to convert the angle to be between -180 to 180 degree
    return angle % 360

def angle_range_convert(angle): # input is [0,360] the car need to spin
    angle = angle % 360
    
    if angle <= 180 :
        return angle
    else:
        return (angle  - 360) 

In [118]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [119]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)
display(widgets.HBox([image, blocked_slider]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [143]:
from jetbot import Robot

robot = Robot()
robot.stop()

curr_x = 0          #use to keep track on the x of the car current status
curr_y = 0          #use to keep track on the y of the car current status
curr_angle = 0      #use to keep track on the angle of the car current status, range from -180 to 180. 0 indicate straight
des_x = 4
des_y = 4
spin_around_time = 3.1
ended = False

In [144]:
import torch.nn.functional as F
import time
import random


no_obj_go_time = 0.001
obj_spin_time_max = 1
obj_go_time = 2
go_forward_ct = 10000000
go_forward_before_redirect_max = 200 # #* 0.001s
start_time = None
last_change = -1 # 0 is left, 1 is forward, 2 is right

def update(change):
    global blocked_slider, robot, curr_x, curr_y, curr_angle 
    global des_x, des_y, spin_time
    global go_forward_ct, start_time, last_change, ended 
    
    if ended:
        return
    
    #if not des_x or not des_y:
    #    print("hahaha")
    
    if reach_destination(curr_x, des_x, curr_y, des_y):
        #camera.unobserve(update, names='value')
        robot.stop()
        print("stop")
        camera.unobserve(update, names='value')
        robot.stop()
        ended = True
        return
        
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    print("current location", curr_x, curr_y, curr_angle )
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    # takes only 0.05s
    '''
    if last_change == 0:
        # print("time", time.time()-start_time)
        curr_angle = check_angle(curr_angle + (360 * (time.time()-start_time)/ spin_around_time) )  #make sure the angle is between -180 to 180 degree
    elif last_change == 1:
        # print("time", time.time()-start_time)
        curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, (time.time()-start_time)) 
    '''
    
    
    if prob_blocked < 0.5: #if no object
        if start_time:
            curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, time.time() - start_time) 
        start_time = None
        target_angle = convert_angle(des_x - curr_x, des_y - curr_y)
        print("target", target_angle)
        angle_diff = angle_range_convert(target_angle - curr_angle) 
        print("angle diff",angle_diff)
        #if go_forward_ct > random.randint(6,9) * 0.1 * go_forward_before_redirect_max and abs(angle_diff) > 2: 
        if  abs(angle_diff) > 2:
            
                #not pointing at the correct angle
                #adjust to the left while going straight if angle is negative and right if angle is positive
            #straight()
            #...
            if angle_diff < 0: # target < current
                spin_right()
            else:
                spin_left()

            time_to_turn = spin_around_time * abs(angle_diff) / 360     #calculate how long does the spin need to be
            print("time_to_turn",time_to_turn)
            #time_to_turn = 0.001
            sleep(time_to_turn)     #sleep to wait until the car turn enough
            robot.stop()
            
            #curr_angle = target_angle    #x and y did not change, only the angle change to the target value in this case.
            
            #print("current location", curr_x, curr_y, curr_angle )
            
            if angle_diff > 0 :
                curr_angle = check_angle(curr_angle + (360 * time_to_turn / spin_around_time) )
            else:
                curr_angle = check_angle(curr_angle - (360 * time_to_turn / spin_around_time) )
            
        else:
            
            #go straight 
            straight()
            start_time = time.time()
            #sleep(no_obj_go_time)
            sleep(0.001)
            # curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, no_obj_go_time)   
            
            #print("current location", curr_x, curr_y, curr_angle )

            
    else:
        if start_time:
            curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, time.time() - start_time) 
        start_time = None
        spin_left()
        obj_spin_time = 0.7
        sleep(obj_spin_time)
        robot.stop()
        #obj_spin_time = 0.001
        curr_angle = check_angle(curr_angle + (360 * obj_spin_time / spin_around_time) )  #make sure the angle is between -180 to 180 degree
      
        straight()
        obj_go_time = 2
        sleep(obj_go_time)
        robot.stop()
        curr_x, curr_y = update_info_straight(curr_x, curr_y,curr_angle,obj_go_time)
        #curr_angle = check_angle(curr_angle + (360 * tmp/ spin_around_time) )
        #curr_x, curr_y = update_info_straight(curr_x, curr_y,curr_angle,obj_go_time)
        #print("current location", curr_x, curr_y, curr_angle )
        
    # time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize
camera.observe(update, names='value')

current location 0 0 0
target 45.0
angle diff 45.0
time_to_turn 0.3875
current location 0 0 45.0
target 45.0
angle diff 0.0
current location 0 0 45.0
target 45.0
angle diff 0.0
current location 0.016464329195992666 0.016464329195992663 45.0
target 45.0
angle diff 0.0
current location 0.0323443513429697 0.0323443513429697 45.0
target 45.0
angle diff 0.0
current location 0.04811123448980409 0.04811123448980409 45.0
target 45.0
angle diff 0.0
current location 0.06259904507802987 0.06259904507802987 45.0
target 45.0
angle diff 0.0
current location 0.07588162438623812 0.07588162438623812 45.0
target 45.0
angle diff 0.0
current location 0.09331059379220873 0.09331059379220873 45.0
target 45.0
angle diff 0.0
current location 0.1125032065034874 0.11250320650348739 45.0
target 45.0
angle diff 0.0
current location 0.1285258535858253 0.12852585358582527 45.0
target 45.0
angle diff 0.0
current location 0.14350182583224366 0.14350182583224363 45.0
target 45.0
angle diff 0.0
current location 0.16051

In [140]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [141]:
camera.unobserve(update, names='value')
robot.stop()

In [142]:
robot.stop()

In [22]:
def funcc(x, y):
    x = x + 1
    y = y + 1
    return x,y

current location 1.3722918623269469 1.3722918623269464 45.0
time 0.07954621315002441
current location 1.3895323847438557 1.3895323847438552 45.0
time 0.057428598403930664
current location 1.4017865472309192 1.4017865472309188 45.0
time 0.06351613998413086
current location 1.415313005063351 1.4153130050633504 45.0
time 0.06566143035888672
current location 1.4295896093643188 1.4295896093643183 45.0
time 0.05716729164123535
current location 1.441778831987197 1.4417788319871965 45.0
time 0.06627583503723145
current location 1.45589424988072 1.4558942498807195 45.0
time 0.0648183822631836
current location 1.4697088910045297 1.4697088910045293 45.0
time 0.0549471378326416
current location 1.4814223937188955 1.481422393718895 45.0
time 0.058904170989990234
current location 1.4939778893140727 1.4939778893140723 45.0
time 0.06731343269348145
current location 1.508323378424247 1.5083233784242465 45.0
time 0.05993819236755371
current location 1.5211030278185445 1.521103027818544 45.0
time 0.06142

In [ ]:
a = 1
b = 1
a, b = funcc(a, b)

In [ ]:
print(a)

In [ ]:
import math
from time import sleep
def convert_angle(delta_x,delta_y):
    if(delta_x == 0):      #special case when x = 0 
        if(delta_y == 0):          #reach the destination
            return None
        elif(delta_y > 0):     
            return 90
        else:
            return -90
    temp = delta_y / delta_x
    
    if(delta_x > 0): #in case from -90 to 90 degree
        return math.degrees(math.atan(temp))
    else:
        
        if(delta_y > 0):  #case for 90 to 180 degree
            return 180 + math.degrees(math.atan(temp))
        else:       #case for -180 to -90 degree
            return math.degrees(math.atan(temp)) + 180 - 360
        
print(convert_angle(3 , 3))
print(convert_angle(3 , -3))
print(convert_angle(-3 , 3))
print(convert_angle(-3 , -3))

In [ ]:
def update_info_straight(curr_x,curr_y,curr_angle, time):    #for keep space
    speed = 1       #currently defalut 1, can use calibration to change that
    curr_x = curr_x + math.cos(math.radians(curr_angle)) * time * speed
    curr_y = curr_y + math.sin(math.radians(curr_angle)) * time * speed

    return curr_x,curr_y

x, y = update_info_straight(3, 3, 45, 1.414)
print(x, y)
x, y = update_info_straight(3, 3, -45, 1.414)
print(x, y)
x, y = update_info_straight(3, 3, 135, 1.414)
print(x, y)
x, y = update_info_straight(3, 3, -135, 1.414)
print(x, y)